In [5]:
import dotenv
import os
apikey=os.environ["pinecone_api"]

In [6]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [22]:
from pinecone import Pinecone,ServerlessSpec

index_name="hybrid-search-uhi-mitigation"

## initialize the pinecone client 

pc=Pinecone(api_key=apikey)

# ## creating the index

# if index_name not in pc.list_indexes().names():
#     pc.create_index(
#         name=index_name,
#         dimension=384,# hugging face embeddings convert text to default 384 dimension
#         metric="dotproduct", # sparse values support only dot product 
#         spec=ServerlessSpec(cloud="aws",region="us-east-1"),
#     )

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [4]:
# for sparse matrix we will use the BM25Encoder which uses TF-IDF by default

from pinecone_text.sparse import BM25Encoder

sparse_encoder=BM25Encoder().default()

/Users/prithivi/Desktop/UHI_mitigation_hybrid_search_rag/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_community.document_loaders import PyPDFLoader
import os
# Folder where all the PDF files are stored
pdf_folder = 'data/'

# Get all PDF file paths in the folder
pdf_files = [os.path.join(pdf_folder, file) for file in os.listdir(pdf_folder) if file.endswith('.pdf')]

# Load all documents
docs = []
for file_path in pdf_files:
    loader = PyPDFLoader(file_path)
    docs.extend(loader.load())

In [15]:
len(docs)

586

In [10]:
# Extract text from each document (split by pages)
corpus = [doc.page_content for doc in docs]

In [11]:
corpus

['Nature Cities | Volume 1 | August 2024 | 522–532 522\nnature cities\nArticle\nhttps://doi.org/10.1038/s44284-024-00091-z\nMitigating urban heat island through \nneighboring rural land cover\nMiao Yang1,2, Chen Ren\u2009  \u20091,2, Haorui Wang1,2, Junqi Wang\u2009  \u20091,2, Zhuangbo Feng1,2, \nPrashant Kumar1,3,4, Fariborz Haghighat1,5 & Shi-Jie Cao\u2009  \u20091,2,3 \nGlobally, the deteriorating Urban Heat Island (UHI) effect poses a significant \nthreat to human health and undermines ecosystem stability. UHI mitigation \nstrategies have been investigated and utilized extensively within cities by \nthe provision of green, blue or gray infrastructures. However, urban land is \nprecious and limited for these interventions, making it challenging to address \nthis issue. Neighboring rural land cover may serve as a cooling source and have \na great potential to mitigate UHI through processes such as heat absorption \nand circulation. This study aims to address the following questions:

In [13]:
 pip install tiktoken

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/4d/ae/4613a59a2a48e761c5161237fc850eb470b4bb93696db89da51b79a871f1/tiktoken-0.9.0-cp311-cp311-macosx_10_12_x86_64.whl.metadata
  Using cached tiktoken-0.9.0-cp311-cp311-macosx_10_12_x86_64.whl.metadata (6.7 kB)
Using cached tiktoken-0.9.0-cp311-cp311-macosx_10_12_x86_64.whl (1.1 MB)

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: /Users/prithivi/Desktop/UHI_mitigation_hybrid_search_rag/venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",  # Split by lines or sentences
    chunk_size=800,  # Adjust based on your needs
    chunk_overlap=200,
)
sentences = text_splitter.split_text("\n".join(corpus))

In [18]:
len(sentences)

680

In [20]:
## tfidf values on these sentence
sparse_encoder.fit(sentences)

## store the values to a json file
sparse_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 680/680 [00:04<00:00, 168.89it/s]


In [23]:
index=pc.Index(index_name)
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [24]:
retriever.add_texts(sentences)

100%|██████████| 22/22 [00:20<00:00,  1.07it/s]


[Document(metadata={'score': 0.56877178}, page_content="Urban heat island mitigation strategie s and climate in Montreal\nPLOS Climate | https://doi.or g/10.137 1/journal.pclm .0000196 June 26, 2023 25 / 25\nA Study on Urban Heat Island (UHI): Challenges and \nOpportunities for Mitigation\nANITA YADAV* and JASWANT SINGH \nDepartment of Environmental Sciences, Dr. Ram Manohar Lohia Avadh University, \nAyodhya, Uttar Pradesh.\nAbstract\nUrban Heat Island (UHI) is a significant issue in metropolitan regions in India, \nwhere cities are experiencing increasing temperatures ranging from 1 to  \n5 °C. Although UHI is a global phenomenon, it particularly impacts major \ncities in India, and there is a need for more research in this field. This review \ncompares the latest global developments in UHI research with the current \nresearch in India, highlighting areas where India may lag. The review notes that  \nthere is a lack of research on UHI in numerical modeling in India, which is \na limit